In [ ]:
%pip install openai

In [3]:
import json
from openai import OpenAI

In [4]:
client = OpenAI(api_key="sk-32e5e7a6f0a74aea99fc06a0b0a670f4", base_url="https://api.deepseek.com")

In [5]:
raw_book = ""
with open('killers.txt', 'r', encoding='utf-8') as file:
    line_number = 1
    for line in file:
        raw_book += f"{line_number}: {line}"
        line_number += 1
print(raw_book)

gold = ""
with open('killers_gold_tags.txt', 'r', encoding='utf-8') as file:
    line_number = 1
    for line in file:
        gold += f"{line_number}: {line}"
        line_number += 1
print(gold)

1: The door of Henry’s lunchroom opened and two men came in.
2: They sat down at the counter.
3: “What’s yours?” George asked them.
4: “I don’t know,” one of the men said.
5: “What do you want to eat, Al?”
6: “I don’t know,” said Al.
7: “I don’t know what I want to eat.”
8: Outside it was getting dark.
9: The streetlight came on outside the window.
10: The two men at the counter read the menu.
11: From the other end of the counter Nick Adams watched them.
12: He had been talking to George when they came in.
13: “I’ll have a roast pork tenderloin with apple sauce and mashed potatoes,” the first man said.
14: “It isn’t ready yet.”
15: “What the hell do you put it on the card for?”
16: “That’s the dinner,” George explained.
17: “You can get that at six o’clock.”
18: George looked at the clock on the wall behind the counter.
19: “It’s five o’clock.”
20: “The clock says twenty minutes past five,” the second man said.
21: “It’s twenty minutes fast.”
22: “Oh, to hell with the clock,” the firs

In [ ]:
prompt = f"""For the sentences in the following text, tag the sentences with tags indicating characters, places and speakers. Add tags for characters and places even if they are not directly mentioned, but it is implied that they are related to the sentence, for instance if they are addressed by a speaker or they are referred to in the sentence. For instance, if the sentence talks of a man without mentioning his name, look in the wider context of text who the man is and tag the sentence with his tag. The valid character tags are the following: {", ".join(dr_data["characters"])}. The valid place tags are the following: {", ".join(dr_data["places"])}. The valid speaker tags are any character tag postfixed with "line", e.g. "{dr_data["characters"][0]} line", "{dr_data["characters"][1]} line" etc. Output the lines {start}-{start+length}. Make sure to preserve quotation marks in the sentences. Give the output in JSON with the following structure: {example}"""


In [39]:
def tag_lines_iterative(dr_data, text, generated_output):
    # TODO: pick characters from dr_data
    example = '{1: {"sentence": "“How is it going” George asked them.", "tags": {"speakers": ["George"], "addressees": ["Al","Max"],"mentioned_characters": [], "other_present_characters": ["Nick"], "place": "Lunchroom"}, 2: {"sentence": "A woman was washing a shirt that belonged to the man.", "tags": {"speakers": [], "addressees": [],"mentioned_characters": ["Mrs. Bell","Ole"],"other_present_characters": [], "place": "Lunchroom"}}\n'

    if generated_output:
        unclosed_generated_output = json.dumps(generated_output,ensure_ascii=False)[:-1]
    else:
        unclosed_generated_output = ""

    prompt = f"""For each of the sentences in the following text, tag the sentences with tags indicating characters, places and speakers. When tagging sentences, follow these rules. 1. Add tags for characters and places even if they are not directly mentioned, but it is implied that they are related to the sentence. 2. If the sentence contains a line of dialogue, tag the persons who are being addressed by the speaker in addition to the speaker. 3. If the sentence refers to persons without naming them (for instance by using pronouns or descriptions, such as "the man"). look in the wider context of the text who is being referred to, and tag the sentence with their tag. The valid character tags are {", ".join(dr_data["characters"][:-1])}, and {dr_data["characters"][-1]}. The valid place tags are {", ".join(dr_data["places"][:-1])}, and {dr_data["places"][-1]}. Output exactly the same amount of lines as in the original text. Make sure to include all revelant tags for every sentence. Output the sentences and tags in JSON in the following format:""" + example + text + unclosed_generated_output

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "user", "content": prompt},
        ],
        response_format={
        'type': 'json_object'
        }, temperature=0,max_tokens=500,
        stream=False
    )

    return response.choices[0].message.content
    

In [37]:
def tag_lines(text, dr_data, start, length, window=20):
    #example = """{LINE NUMBER: {"sentence": SENTENCE, "tags": LIST OF TAGS}}\n"""
    #example = """“What’s yours?” George asked them. ||| George, Al, Max, George line, Lunchroom"""
    example = '{1: {"sentence": "“How is it going” George asked them.", "tags": {"speakers": ["George"], "addressees": ["Al","Max"],"mentioned_characters": [], "other_present_characters": ["Nick"], "place": "Lunchroom"}, 2: {"sentence": "A woman was washing a shirt that belonged to the man.", "tags": {"speakers": [], "addressees": [],"mentioned_characters": ["Mrs. Bell","Ole"],"other_present_characters": [], "place": "Lunchroom"}}'

    prompt = f"""For each of the sentences in the following text, tag the sentences with tags indicating characters, places and speakers. When tagging sentences, follow these rules. 1. Add tags for characters and places even if they are not directly mentioned, but it is implied that they are related to the sentence. 2. If the sentence contains a line of dialogue, tag the persons who are being addressed by the speaker in addition to the speaker. 3. If the sentence refers to persons without naming them (for instance by using pronouns or descriptions, such as "the man"). look in the wider context of the text who is being referred to, and tag the sentence with their tag. The valid character tags are {", ".join(dr_data["characters"][:-1])}, and {dr_data["characters"][-1]}. The valid place tags are {", ".join(dr_data["places"][:-1])}, and {dr_data["places"][-1]}. The valid speaker tags are any character tag postfixed with "line", e.g. "{dr_data["characters"][0]} line", "{dr_data["characters"][1]} line" etc. Output exactly the same amount of lines as in the original text. Make sure to include all revelant tags for every sentence. Output sentences {start}-{start+length}. Output the sentences and tags in JSON in the following format:""" + example

    print(prompt)

    # only retain some text outside of the window were are interested in
    split_text = text.split("\n")
    if start-window > 0:
        text_start = start-window
    else:
        text_start = 0
    if start+length+window > len(split_text):
        text_end = len(split_text)
    else:
        text_end = start+length+window
    truncated_text = "\n".join(split_text[text_start:text_end])
    print(truncated_text)
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "user", "content": prompt + truncated_text},
        ],
        response_format={
        'type': 'json_object'
        }, temperature=0,
        stream=False
    )

    json_data = json.loads(response.choices[0].message.content)
    return json_data


In [40]:
dr_data = {
    "characters": ["Max", "Al", "George", "Nick", "Sam", "Ole", "Mrs. Bell", "Motorman"], 
    "places": ["Lunchroom", "Boarding house"]
    }
output = None
output = tag_lines_iterative(dr_data,raw_book,output)
print(output)

{
  "1": {
    "sentence": "The door of Henry’s lunchroom opened and two men came in.",
    "tags": {
      "speakers": [],
      "addressees": [],
      "mentioned_characters": ["Al", "Max"],
      "other_present_characters": ["George", "Nick"],
      "place": "Lunchroom"
    }
  },
  "2": {
    "sentence": "They sat down at the counter.",
    "tags": {
      "speakers": [],
      "addressees": [],
      "mentioned_characters": ["Al", "Max"],
      "other_present_characters": ["George", "Nick"],
      "place": "Lunchroom"
    }
  },
  "3": {
    "sentence": "“What’s yours?” George asked them.",
    "tags": {
      "speakers": ["George"],
      "addressees": ["Al", "Max"],
      "mentioned_characters": [],
      "other_present_characters": ["Nick"],
      "place": "Lunchroom"
    }
  },
  "4": {
    "sentence": "“I don’t know,” one of the men said.",
    "tags": {
      "speakers": ["Al"],
      "addressees": ["George"],
      "mentioned_characters": [],
      "other_present_characters

In [32]:
print(json.dumps(output,ensure_ascii=False)[:-1])

{"1": {"sentence": "The door of Henry’s lunchroom opened and two men came in.", "tags": {"speakers": [], "addressees": [], "other_present_characters": ["Al", "Max"], "place": "Lunchroom"}}, "2": {"sentence": "They sat down at the counter.", "tags": {"speakers": [], "addressees": [], "other_present_characters": ["Al", "Max"], "place": "Lunchroom"}}, "3": {"sentence": "“What’s yours?” George asked them.", "tags": {"speakers": ["George line"], "addressees": ["Al", "Max"], "other_present_characters": ["Nick"], "place": "Lunchroom"}}, "4": {"sentence": "“I don’t know,” one of the men said.", "tags": {"speakers": ["Al line"], "addressees": ["George"], "other_present_characters": ["Max", "Nick"], "place": "Lunchroom"}}, "5": {"sentence": "“What do you want to eat, Al?”", "tags": {"speakers": ["Max line"], "addressees": ["Al"], "other_present_characters": ["George", "Nick"], "place": "Lunchroom"}}, "6": {"sentence": "“I don’t know,” said Al.", "tags": {"speakers": ["Al line"], "addressees": ["

In [21]:
dr_data = {
    "characters": ["Max", "Al", "George", "Nick", "Sam", "Ole", "Mrs. Bell", "Motorman"], 
    "places": ["Lunchroom", "Boarding house", "Lunchroom kitchen"]
    }

tag_json = None

batch_size = 50
batch_number = 0
len_raw_book = len(raw_book.split("\n"))
while batch_number * batch_size < len_raw_book:
    start = batch_number * batch_size + 1 
    
    if start + batch_size > len_raw_book:
        effective_batch_size = len_raw_book - start
    else:
        effective_batch_size = batch_size - 1
    batch_tag_json = tag_lines(raw_book, dr_data, start, effective_batch_size)
    if tag_json:
        tag_json.update(batch_tag_json)
    else:
        tag_json = batch_tag_json
    batch_number += 1
    if batch_number == 1:
        break

For each of the sentences in the following text, tag the sentences with tags indicating characters, places and speakers. When tagging sentences, follow these rules. 1. Add tags for characters and places even if they are not directly mentioned, but it is implied that they are related to the sentence. 2. If the sentence contains a line of dialogue, tag the persons who are being addressed by the speaker in addition to the speaker. 3. If the sentence refers to persons without naming them (for instance by using pronouns or descriptions, such as "the man"). look in the wider context of the text who is being referred to, and tag the sentence with their tag. The valid character tags are Max, Al, George, Nick, Sam, Ole, Mrs. Bell, and Motorman. The valid place tags are Lunchroom, Boarding house, and Lunchroom kitchen. The valid speaker tags are any character tag postfixed with "line", e.g. "Max line", "Al line" etc. Output exactly the same amount of lines as in the original text. Make sure to i

In [23]:
print(tag_json)

{'1': {'sentence': 'The door of Henry’s lunchroom opened and two men came in.', 'tags': ['Lunchroom', 'Al', 'Max']}, '2': {'sentence': 'They sat down at the counter.', 'tags': ['Al', 'Max', 'Lunchroom']}, '3': {'sentence': '“What’s yours?” George asked them.', 'tags': ['George', 'Al', 'Max', 'George line', 'Lunchroom']}, '4': {'sentence': '“I don’t know,” one of the men said.', 'tags': ['Al', 'Max', 'Lunchroom']}, '5': {'sentence': '“What do you want to eat, Al?”', 'tags': ['Max', 'Al', 'Max line', 'Lunchroom']}, '6': {'sentence': '“I don’t know,” said Al.', 'tags': ['Al', 'Al line', 'Lunchroom']}, '7': {'sentence': '“I don’t know what I want to eat.”', 'tags': ['Al', 'Al line', 'Lunchroom']}, '8': {'sentence': 'Outside it was getting dark.', 'tags': ['Lunchroom']}, '9': {'sentence': 'The streetlight came on outside the window.', 'tags': ['Lunchroom']}, '10': {'sentence': 'The two men at the counter read the menu.', 'tags': ['Al', 'Max', 'Lunchroom']}, '11': {'sentence': 'From the othe

In [7]:
def compare_letter_sequence(str1, str2):
    """
    Compare two strings to see if they have the same letters in the same order,
    ignoring all non-letter characters (using isalpha()).
    
    Args:
        str1 (str): First string to compare
        str2 (str): Second string to compare
        
    Returns:
        bool: True if the letter sequences match, False otherwise
    """
    # Extract letters from each string
    letters1 = [c.lower() for c in str1 if c.isalpha()]
    letters2 = [c.lower() for c in str2 if c.isalpha()]
    
    return letters1 == letters2

# Example usage:
print(compare_letter_sequence("Hello, World!", "h e l l o w o r l d"))  # True
print(compare_letter_sequence("Python3", "P y t h o n"))  # True
print(compare_letter_sequence("abc", "a b c d"))  # False
print(compare_letter_sequence("Test", "t e s t"))  # True
print(compare_letter_sequence("Case", "c a s e"))  # True
print(compare_letter_sequence("Different", "Letters"))  # False

True
True
False
True
True
False


In [11]:
def calculate_f1_score(test_list, gold_list):
    test_set = set(test_list)
    gold_set = set(gold_list)

    true_positives = len(test_set & gold_set)
    predicted_positives = len(test_set)
    actual_positives = len(gold_set)

    precision = true_positives / predicted_positives if predicted_positives > 0 else 0
    recall = true_positives / actual_positives if actual_positives > 0 else 0

    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)

    return precision, recall, f1

def macro_f1(test_lists, gold_lists):
    f1s = []
    for test, gold in zip(test_lists, gold_lists):
        _, _, f1 = calculate_f1_score(test, gold)
        f1s.append(f1)
    return sum(f1s) / len(f1s) if f1s else 0

def micro_f1(test_lists, gold_lists):
    total_tp = total_pred = total_gold = 0
    for test, gold in zip(test_lists, gold_lists):
        test_set = set(test)
        gold_set = set(gold)
        total_tp += len(test_set & gold_set)
        total_pred += len(test_set)
        total_gold += len(gold_set)
    
    precision = total_tp / total_pred if total_pred > 0 else 0
    recall = total_tp / total_gold if total_gold > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    return f1


In [26]:
import ast
tags = [set(item["tags"]) for item in tag_json.values()]
print(tags)
gold_tags = [set(ast.literal_eval(x.split(" ||| ")[1])) for x in gold.split("\n")]
print(gold_tags)
print(set(gold_tags[0]))
for (sentence,sent_tags,sent_gold_tags) in zip([f'{item["sentence"]}' for item in tag_json.values()],tags,gold_tags):
    #print(sent_tags)
    #print(sent_gold_tags)
    too_many_tags = set(sent_tags) - set(sent_gold_tags)
    not_all_tags = set(sent_gold_tags) - set(sent_tags)
    print(f"{sentence}: extra tags: {too_many_tags}. missing tags: {not_all_tags} gold tag: {sent_gold_tags}")
print(macro_f1(tags,gold_tags))



[{'Al', 'Max', 'Lunchroom'}, {'Al', 'Max', 'Lunchroom'}, {'Lunchroom', 'Al', 'George line', 'Max', 'George'}, {'Al', 'Max', 'Lunchroom'}, {'Max', 'Lunchroom', 'Al', 'Max line'}, {'Al', 'Al line', 'Lunchroom'}, {'Al', 'Al line', 'Lunchroom'}, {'Lunchroom'}, {'Lunchroom'}, {'Al', 'Max', 'Lunchroom'}, {'Al', 'Nick', 'Max', 'Lunchroom'}, {'Nick', 'Lunchroom', 'George'}, {'Max', 'Lunchroom', 'Max line'}, {'Lunchroom', 'George line', 'George'}, {'Max', 'Lunchroom', 'Max line'}, {'Lunchroom', 'George line', 'George'}, {'Lunchroom', 'George line', 'George'}, {'Lunchroom', 'George'}, {'Lunchroom', 'George line', 'George'}, {'Al', 'Al line', 'Lunchroom'}, {'Lunchroom', 'George line', 'George'}, {'Max', 'Lunchroom', 'Max line'}, {'Max', 'Lunchroom', 'Max line'}, {'Lunchroom', 'George line', 'George'}, {'Lunchroom', 'George line', 'George'}, {'Max', 'Lunchroom', 'Max line'}, {'Lunchroom', 'George line', 'George'}, {'Max', 'Lunchroom', 'Max line'}, {'Max', 'Lunchroom', 'Max line'}, {'Lunchroom', 'G

In [20]:
# Extract sentences
sentences = [f'{item["sentence"]} ||| {item["tags"]}' for item in tag_json.values()]


print(tag_json)
# Print the sentences
for sentence in sentences:
    print(sentence)

print(len(sentences))
for line in raw_book.split("\n"):
    if sentences and compare_letter_sequence(line,sentences[0]):
        sentences.pop(0)
print(len(sentences))


{'1': {'sentence': 'The door of Henry’s lunchroom opened and two men came in.', 'tags': ['Lunchroom', 'Max', 'Al']}, '2': {'sentence': 'They sat down at the counter.', 'tags': ['Lunchroom', 'Max', 'Al']}, '3': {'sentence': '“What’s yours?” George asked them.', 'tags': ['George line', 'Max', 'Al']}, '4': {'sentence': '“I don’t know,” one of the men said.', 'tags': ['Max line', 'Al']}, '5': {'sentence': '“What do you want to eat, Al?”', 'tags': ['Max line', 'Al']}, '6': {'sentence': '“I don’t know,” said Al.', 'tags': ['Al line', 'Max']}, '7': {'sentence': '“I don’t know what I want to eat.”', 'tags': ['Al line', 'Max']}, '8': {'sentence': 'Outside it was getting dark.', 'tags': ['Lunchroom']}, '9': {'sentence': 'The streetlight came on outside the window.', 'tags': ['Lunchroom']}, '10': {'sentence': 'The two men at the counter read the menu.', 'tags': ['Lunchroom', 'Max', 'Al']}, '11': {'sentence': 'From the other end of the counter Nick Adams watched them.', 'tags': ['Lunchroom', 'Nick